# Assignment 2: Milestone I Natural Language Processing
## Task 1. Basic Text Pre-processing
#### Student Name: Rylan Pease
#### Student ID: s3896416

Date: 1/10/23

Version: 1.0

Environment: Python 3 and Jupyter notebook

Libraries used: please include all the libraries you used in your assignment, e.g.,:
* pandas
* re
* numpy
* os
* collections

## Introduction
In this doccument I will be pre-processing the data as well as creating the vocab.txt file

<span style="color: red"> Note that this is a sample notebook only. You will need to fill in the proper markdown and code blocks. You might also want to make necessary changes to the structure to meet your own needs. Note also that any generic comments written in this notebook are to be removed and replace with your own words.</span>

## Importing libraries 

In [2]:
# Code to import libraries as you need in this assessment, e.g.,
import numpy as np
import pandas as pd
import re as re
import os
from collections import defaultdict, Counter

### 1.1 Examining and loading data
- xamine the data folder, including the categories and job advertisment txt documents, etc. Explain your findings here, e.g., number of folders and format of txt files, etc.
- Load the data into proper data structures and get it ready for processing.
- Extract webIndex and description into proper data structures.


In [3]:
# Code to inspect the provided data file...

sub_dirs = os.listdir('data')
print('Total Subdirectories: ' + str(len(sub_dirs)))

dataFiles = []
for i in sub_dirs:
    inserting = []
    inserting.extend(os.listdir('data/'+i))
    dataFiles.append(inserting)

dataDirs = []
for i in range(len(dataFiles)):
    for j in range(len(dataFiles[i])):
        dataDirs.append('data/'+sub_dirs[i]+'/'+dataFiles[i][j])
    
print('Total Data Points: ' + str(len(dataDirs)))

# Code to read in the data file...

textList = []

for i in range(len(dataDirs)):
    with open(dataDirs[i], 'r', encoding='utf-8') as file:
        text = file.read()
        textList.append(text + "\nCategory: " + dataDirs[i].split('/')[1])

data_list = []

for block in textList:
    lines = block.strip().split("\n")
    data = {}
    for line in lines:
        key, value = line.split(": ", 1)
        data[key] = value
    data_list.append(data)


df = pd.DataFrame(data_list)



Total Subdirectories: 4
Total Data Points: 776


In [4]:

# set Title to string
df['Title'] = df['Title'].astype(str)

# set Webindex to int
df['Webindex'] = df['Webindex'].astype(int)

# set Company as string
df['Company'] = df['Company'].astype(str)

# set Description as string
df['Description'] = df['Description'].astype(str)

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 776 entries, 0 to 775
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        776 non-null    object
 1   Webindex     776 non-null    int32 
 2   Company      776 non-null    object
 3   Description  776 non-null    object
 4   Category     776 non-null    object
dtypes: int32(1), object(4)
memory usage: 27.4+ KB
None


### 1.2 Pre-processing data
Perform the required text pre-processing steps.

...... Sections and code blocks on basic text pre-processing


<span style="color: red"> You might have complex notebook structure in this section, please feel free to create your own notebook structure. </span>

In [5]:
# Regular expression for word tokenization
token_pattern = r"[a-zA-Z]+(?:[-'][a-zA-Z]+)?"

# Tokenize the description column after adding to the DataFrame
df['Tokenized Description'] = df['Description'].apply(lambda x: [word.lower() for word in re.findall(token_pattern, x) if len(word) > 1])

with open("stopwords_en.txt", "r") as f:
    stopwords = set(f.read().splitlines())

# Remove stopwords
df['Tokenized Description'] = df['Tokenized Description'].apply(lambda x: [word for word in x if word not in stopwords])

# Count term frequencies across all documents
term_frequencies = Counter(word for words in df['Tokenized Description'] for word in words)

# Remove words that appear only once
df['Tokenized Description'] = df['Tokenized Description'].apply(lambda x: [word for word in x if term_frequencies[word] > 1])

# Compute document frequencies
document_frequencies = defaultdict(int)
for words in df['Tokenized Description']:
    for word in set(words):  # Convert to set to ensure each word is counted once per document
        document_frequencies[word] += 1

# Get the top 50 most frequent words based on document frequency
top_50_words = {word for word, freq in Counter(document_frequencies).most_common(50)}

# Remove the top 50 words from each document
df['Tokenized Description'] = df['Tokenized Description'].apply(lambda x: [word for word in x if word not in top_50_words])

# Save the DataFrame to a CSV file
df.to_csv("processed_job_ads.csv", index=False)

## Saving required outputs
Save the vocabulary, bigrams and job advertisment txt as per spectification.
- vocab.txt

In [6]:
# Gather all unique words from the tokenized descriptions
vocabulary = sorted(set(word for words in df['Tokenized Description'] for word in words))

# Assign an integer index to each word
vocab_dict = {word: idx for idx, word in enumerate(vocabulary)}

# Save the vocabulary to a text file
with open("vocab.txt", "w") as f:
    for word, idx in vocab_dict.items():
        f.write(f"{word}:{idx}\n")